# Hash Functions

This section consists of toy hash functions and the cr game. 

In [1]:
from cryptogame.tools.utils import xor
from cryptogame.game.cr import *
from simulate import *
from cryptogame.tools.block_cipher import *

## Hash Example 1
Let $E: \{0, 1\}^k \times \{0,1\}^n \rightarrow \{0,1\}$ be a blockcipher.<br>
Let $H: \{0, 1\}^k \times \{0,1\}^{2n} \rightarrow \{0,1\}$ be defined by:<br>

&nbsp;&nbsp;&nbsp;&nbsp;**Alg** $H(K,x[1]x[2])$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$y \leftarrow E_K(E_K(x[1]) \oplus x[2]);$ Return $y$<br>

First, initialize the game.




In [2]:
key_length = 5 
block_length = 5

## Initialize the block cipher
E = BlockCipher(key_len = key_length, block_len= block_length)


## Hash function as defined above
def H2(K, x):
	'''
	Let E: {0,1}^n * {0,1}^n -> {0,1}^n be a block cipher. 
	Let H: {0,1}^k * {0,1}^2n -> {0,1}^n be a hash function. 
	'''

	key_len, block_len, message_len = key_length, block_length, 2*block_length
	assert len(x) == message_len, f"Expected {message_len} got {len(x)} for messages"
	
	y = E.evaluate(K, xor(E.evaluate(K, x[0:block_len]), x[block_len:]))
	return y

## Define the CR against H2 with key length 5
game = CR(H2, 5)

## Initializing the game. Note the adversary has access to the key k. 
k = game._initialize()


### TODO: You are required to write the function for the adversary 

Let's show that $H$ is not collision-resistant by giving an efficient adversary $A$ such that:<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\textbf{Adv}_H^{cr}(A)=1$<br>

Using the adversary function **adv_H2**$(game)$ below:<br>
&nbsp;&nbsp;&nbsp;&nbsp;Define an efficient adversary that ouptuts two messages $x1$ and $x2$ such that $H(x1) = H(x2)$ <br>

Note: Ensure your return values $x1$ and $x2$ are strings of 0s and 1s as input.<br>
Ex: $x1 =$ '00000'<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$x2 =$ '10101'<br>

Hint: Since the adversary has access to the key, the adversary can compute $E_k$ and $E_k^{-1}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;$E_k(x) = E.evaluate(k,x)$ and <br>
&nbsp;&nbsp;&nbsp;&nbsp;$E_k^{-1}(c) = E.inverse(k,c)$


In [3]:
def adv_H2(game):
	'''
	This is the adversary against H2.
	Define and efficient adversary that ouptuts two messages x1 and x1 such that H(x1) = H(x2)
	'''	
	#========= Your Code goes here =======================
	x_1 = '0'*5 + '1'*5
	x2_2 = '0'*5

	x2_1 = E.inverse(k, xor(xor(E.evaluate(k, x_1[0:5]), x_1[5:]), x2_2))
	x_2 = x2_1 + x2_2
	# =====================================================
	return x_1, x_2

x1, x2 = adv_H2(game)

print("Finalize Returned", game.finalize(x1,x2))

Finalize Returned True


## Hash Example 2: Keyless Hash Function
We say that H: Keys * D -> R is keyless if Keys = {$\epsilon$}consists of just one key, the empty string.<br>

Let $E: \{0, 1\}^b \times \{0, 1\}^n \rightarrow \{0, 1\}^n$ be a block cipher.<br>
Let us define keyless compression function $h: \{0, 1\}^{b+n} \rightarrow \{0, 1\}^n$ by:<br>
&nbsp;&nbsp;&nbsp;&nbsp;$h(x||v)=E_x(v)$<br>

**Question:** Is $h$ collision resistant?<br>

First, we initialize the game.

In [4]:
b = 5  ## change as required
n = 7  ## change as required

E = BlockCipher(key_len=b, block_len=n)

def H3(K, x):
	'''
	Let E: {0,1}^n * {0,1}^n -> {0,1}^n be a block cipher and 
	Let Let H: {0,1}^b+n  -> {0,1}^n be a keyless hash function defined as h(x||v) == E_x(v)
	'''

	key = x[0:b]
	v = x[b:]
	
	assert len(key) == b 
	assert len(v) == n

	return E.evaluate(key, v)


## Let's initiailze the game
game = CR(H3, key_len=b)
k = game._initialize()


### TODO: Hash Function 2 Adversary: 
We seek an adversary that outputs distinct $x_1||v_1,x_2||v_2$ satisfying:<br>
&nbsp;&nbsp;&nbsp;&nbsp;$E_{x_1}(v_1)=E_{x_2}(v_2)$<br>

Note: Ensure your return values $x1$ and $x2$ are strings of 0s and 1s.<br>

In [5]:

def adv_H3(game):

	#========= Your Code goes here =======================
	x1 = '0'*b
	x2 = '1'*b
	v1 = '0'*n
	y = E.evaluate(key=x1, plaintext=v1)
	v2 = E.inverse(key= x2, ciphertext=y)
	x1 += v1
	x2 += v2
	# =====================================================
	return x1, x2

x1, x2 = adv_H3(game)
print("Finalize Returned", game.finalize(x1,x2))

Finalize Returned True


### Inline Question #1:
Is $h$ collision resistant?<br>

**Your Response**:

## Hash Example 3: (From HW 4 Problem 1)

Let $D$ be the set of all strings whose length is a positive multiple of 128. <br>
Define hash function $H: \{0,1\}^{128}\times D\rightarrow \{0,1\}^{128}$ as follows:<br>
**Algorithm** $H_K(M)$:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Parse $M$ as $M[1]M[2]...M[m]$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$C[0] \leftarrow 0^{128}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For $i=1$ to $m$ do:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$B[i]\leftarrow \textbf{AES}_K(C[i-1]\oplus M[i])$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$C[i]\leftarrow \textbf{AES}_K(B[i]\oplus M[i])$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Return $C[M]$<br>


In $H_K$, we parse $M$ has consisting of $m$ blocks of 128-bits each.

First, initialize the game:

In [6]:
from cryptogame.tools.AES import *

key_len = 128
output_len = 128
aes = AES(key_len, block_len=128)

def H_K(K, M):
	c_0 = '0'*128
	m = len(M)// key_len
	assert m*key_len == len(M), "Message length has to be a multiple of 128"

	for i in range(m):
		B_i = aes.evaluate(K, xor(c_0, M[i*len(K): (i+1)*len(K)]))
		C_i = aes.evaluate(K, xor(B_i, M[i*len(K): (i+1)*len(K)]))
		c_0 = C_i
	return C_i

game = CR(H_K, key_len = 128)



### TODO: Hash Function 3 Adversary:

Show that $H$ is not collision resistant by giving a practical adversary $A$ below such that its advantage $\textbf{Adv}_H^{cr}(A)$ is high.<br>

Note: Your adversary should break the hash function without breaking the underlying blockcipher as a pseudorandom function.<br>

Note: Ensure your return values $x1$ and $x2$ are strings of 0s and 1s as input.<br>

In [7]:
def adv_H_K(game):
	key = game._initialize()
	#========= Your Code goes here =======================
	x1 = aes.inverse( key, '0'*128)
	x2 = aes.inverse( key, '0'*128) + aes.inverse(key, '0'*128)
	# =====================================================
	return x1, x2

x1, x2 = adv_H_K(game)

print("Finalize Returned", game.finalize(x1,x2))





Finalize Returned True


Pr[CR_H => 1] = 1000 / 1000 = 1.0

Adv(A) = 1.0


In [8]:
## Let simulate the game with the adversary for 1000 iteration. The simulation will compute the Advantage of the adversary

s = Simulate(game, adversary = adv_H_K)
s.simulate_cr(verbose=True)

Pr[CR_H => 1] = 1000 / 1000 = 1.0

Adv(A) = 1.0
